In [1]:
import time
import datetime
import os
import pandas as pd
import numpy as np
from bokeh.io import output_notebook, show#, output_file
from bokeh.layouts import gridplot
from bokeh.plotting import figure
output_notebook()
def plot_config(width: int, height: int, title=None, x_label: str = None,
                y_label: str = None):
    p = figure(title=title, background_fill_color="#fafafa", width=width, height=height)
    p.output_backend = "svg"  # save plot in HTML as SVG
    p.title.text_font_size = "15pt"
    p.xaxis.axis_label = x_label
    p.yaxis.axis_label = y_label
    p.yaxis.axis_label_text_font_size = "12pt"
    p.yaxis.major_label_text_font_size = "12pt"
    p.xaxis.major_label_text_font_size = "12pt"
    p.xaxis.axis_label_text_font_size = "12pt"
    return p


Loading BokehJS ...

In [2]:
# load data files
df_command = pd.read_csv(r'./data/data.hamster2__command.csv')
df_imu = pd.read_csv(r'./data/data.hamster2__imu.csv')
df_velocity = pd.read_csv(r'./data/data.hamster2__velocity.csv')
df_tf = pd.read_csv(r'./data/data.tf.csv')

# extract useful data
df_command = df_command.iloc[:,[0,3,5,7]]
df_imu = df_imu.iloc[:,[0,3,18,19,20,30,31,32]]
df_velocity = df_velocity.iloc[:,[0,3]]
df_tf = df_tf.iloc[:,[0,3,6,7,8,9,10,11,12]]

# rename the data
df_command.rename(columns={'/hamster2/command/header.stamp': 'command.header.stamp', 
                           '/hamster2/command/drive.steering_angle': 'command.steering_angle',
                           '/hamster2/command/drive.speed': 'command.speed'
                           }, inplace=True)

df_imu.rename(columns={'/hamster2/imu/header.stamp': 'imu.header.stamp', 
                       '/hamster2/imu/angular_velocity.x': 'imu.w_x',
                       '/hamster2/imu/angular_velocity.y': 'imu.w_y',
                       '/hamster2/imu/angular_velocity.z': 'imu.w_z',
                       '/hamster2/imu/linear_acceleration.x': 'imu.a_x',
                       '/hamster2/imu/linear_acceleration.y': 'imu.a_y',
                       '/hamster2/imu/linear_acceleration.z': 'imu.a_z',
                       }, inplace=True)

df_velocity.rename(columns={'/hamster2/velocity/data': 'velocity.v'}, inplace=True)

df_tf.rename(columns={'/tf/transforms.0.header.stamp': 'tf.header.stamp', 
                      '/tf/transforms.0.transform.translation.x': 'tf.x',
                      '/tf/transforms.0.transform.translation.y': 'tf.y',
                      '/tf/transforms.0.transform.translation.z': 'tf.z',
                      '/tf/transforms.0.transform.rotation.x': 'tf.r_x',
                      '/tf/transforms.0.transform.rotation.y': 'tf.r_y',
                      '/tf/transforms.0.transform.rotation.z': 'tf.r_z',
                      '/tf/transforms.0.transform.rotation.w': 'tf.r_w'
                     }, inplace=True)

# transform quaternion rotation to euler angle, and add it to the data
from euler_from_quaternion import *
xyzw = df_tf.iloc[:,5:9].to_numpy()
euler = np.zeros((xyzw.shape[0],3))

for i in range(xyzw.shape[0]):
    euler[i][0], euler[i][1], euler[i][2] = euler_from_quaternion(xyzw[i][0], xyzw[i][1], xyzw[i][2], xyzw[i][3])
df_euler = pd.DataFrame(euler, columns=['tf.roll_x', 'tf.pitch_y', 'tf.yaw_z'])

df_tf = pd.concat([df_tf, df_euler], axis=1)


In [ ]:
df_merge = df_command.merge(df_velocity, how='outer', on='__time')
df_merge = df_merge.merge(df_imu, how='outer', on='__time')
df_merge = df_merge.merge(df_tf, how='outer', on='__time')

df_merge.sort_values(by=['__time'], inplace=True)
df_merge.set_index('__time', inplace=True)
# df_merge.reset_index(drop=True, inplace=True)

# os.makedirs(r'./data', exist_ok=True)  
# df_merge.to_csv(r'./data/data_merge.csv')  
df_merge

In [3]:
df = pd.read_csv(r'./data/data_merge.csv')
data = df.to_numpy()
df.iloc[:,-11:]

,tf.header.stamp,tf.x,tf.y,tf.z,tf.r_x,tf.r_y,tf.r_z,tf.r_w,tf.roll_x,tf.pitch_y,tf.yaw_z
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.682357e+09,2.668901,-3.346754,0.137497,0.005026,0.001024,0.038226,-0.999256,-0.009967,-0.002432,-0.076459
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data[:,-11]

In [4]:
t0 = data[:,0][0]

p = plot_config(width=900, height=600, title='0000000',  x_label='t [ s ]', y_label= 'unknown ')

# p.circle(x=data[:,0]-t0, y=data[:,2]/180*np.pi,legend_label='command steering angle', size=2, fill_color='black', line_color='black')
# p.circle(x=data[:,0]-t0, y=data[:,3],legend_label='command velocity', size=2, fill_color='green', line_color='green')
# p.circle(x=data[:,0]-t0, y=data[:,3]*np.tan(data[:,2]/180*np.pi),legend_label='command w', size=2, fill_color='blue', line_color='blue')

# p.circle(x=data[:,0]-t0, y=data[:,7],legend_label='imu angular velocity w_y', size=2, fill_color='blue', line_color='blue')
# p.circle(x=data[:,5]-t0, y=data[:,11],legend_label='imu linear acceleration a_z', size=2, fill_color='blue', line_color='blue')

# p.circle(x=data[:,0]-t0, y=data[:,4]/-1000,legend_label='vel', size=2, fill_color='red', line_color='red')

p.circle(x=data[:,0]-data[:,0][0], y=data[:,-1],legend_label='tf yaw angle __time', size=3, fill_color='red', line_color='red')
p.circle(x=data[:,-11]-data[:,-11][1], y=data[:,-1],legend_label='tf yaw angle header', size=2, fill_color='blue', line_color='blue')

# output_file(filename="./w_y VS yaw.html", title="KF_all vs tf")
show(p, notebook_handle=True)

In [5]:
p = plot_config(width=900, height=600, title='0000000',  x_label='t [ s ]', y_label= 'unknown ')

p.circle(x=data[:,0]-data[:,0][-10], y=data[:,-1],legend_label='tf yaw angle __time', size=3, fill_color='red', line_color='red')
p.circle(x=data[:,-11]-data[:,-11][-10], y=data[:,-1],legend_label='tf yaw angle header', size=2, fill_color='blue', line_color='blue')

# output_file(filename="./w_y VS yaw.html", title="KF_all vs tf")
show(p, notebook_handle=True)

In [7]:
t0 = data[:,0][0]

p = plot_config(width=900, height=600, title='0000000',  x_label='t [ s ]', y_label= 'unknown ')

# p.circle(x=data[:,0]-t0, y=data[:,2]/180*np.pi,legend_label='command steering angle', size=2, fill_color='black', line_color='black')
# p.circle(x=data[:,0]-t0, y=data[:,3],legend_label='command velocity', size=2, fill_color='green', line_color='green')
# p.circle(x=data[:,0]-t0, y=data[:,3]*np.tan(data[:,2]/180*np.pi),legend_label='command w', size=2, fill_color='blue', line_color='blue')

p.circle(x=data[:,0]-t0, y=data[:,7],legend_label='imu angular velocity w_y', size=2, fill_color='blue', line_color='blue')
# p.circle(x=data[:,5]-t0, y=data[:,11],legend_label='imu linear acceleration a_z', size=2, fill_color='blue', line_color='blue')

# p.circle(x=data[:,0]-t0, y=data[:,4]/-1000,legend_label='vel', size=2, fill_color='red', line_color='red')

p.circle(x=data[:,0]-data[:,0][0], y=data[:,-1],legend_label='tf yaw angle __time', size=3, fill_color='red', line_color='red')
# p.circle(x=data[:,-11]-data[:,-11][1], y=data[:,-1],legend_label='tf yaw angle header', size=2, fill_color='blue', line_color='blue')

# output_file(filename="./w_y VS yaw.html", title="KF_all vs tf")
show(p, notebook_handle=True)